In [3]:
# Import libraries
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json, os, glob

In [4]:
def Model1(data):
    # Create a new model
    m = gp.Model("Model3")

    Blocks = data["Blocks"]
    Sessions = data["Sessions"]
    Sessions_blocks = data["Sessions_b"]
    Interpreters = data["Interpreters"]
    Languages = data["Languages"]
    Languages_sessions = data["Languages_s"]
    Languages_interpreters = data["Languages_i"]

    # Create variables
    x = {}
    for i in Interpreters:
        for b in Blocks:
            for s in Sessions_blocks[b]:
                for l1 in Languages_sessions[s]:
                    if l1 in Languages_interpreters[i]:
                        for l2 in Languages_sessions[s]:
                            if l2 in Languages_interpreters[i] and l2 > l1:
                                print(f"Creating variable for Interpreter {i}, Block {b}, Session {s}, Languages {l1} and {l2}")
                                # Create a binary variable for each interpreter, block, session, and language
                                x[i, b, s, l1, l2] = m.addVar(vtype=GRB.BINARY, name=f"x_{i}_{b}_{s}_{l1}_{l2}")
    
    # Constraints
    # Each interpreter can only be assigned to one language pair per Block
    for i in Interpreters:
        for b in Blocks:
            m.addConstr(gp.quicksum(x[i, b, s, l1, l2] for s in Sessions_blocks[b] for l1 in Languages_sessions[s] for l2 in Languages_sessions[s] if (i, b, s, l1, l2) in x) <= 1, name=f"OneLanguagePairPerSession_{i}_{s}")

    #Only one interpreter per language pair per session
    for b in Blocks:
        for s in Sessions_blocks[b]:
            for l1 in Languages_sessions[s]:
                for l2 in Languages_sessions[s]:
                    if l2 > l1:
                        m.addConstr(gp.quicksum(x[i, b, s, l1, l2] for i in Interpreters if (i, b, s, l1, l2) in x) <= 1, name=f"OneInterpreterPerLanguagePair_{b}_{s}_{l1}_{l2}")
    

    # Set objective function
    m.setObjective(gp.quicksum(x[i, b, s, l1, l2] for i in Interpreters for b in Blocks for s in Sessions_blocks[b] for l1 in Languages_sessions[s] for l2 in Languages_sessions[s] if (i, b, s, l1, l2) in x), GRB.MAXIMIZE)
    return m

In [5]:
with open(r"instances\example.json", "r") as file:
    data_bridge = json.load(file)

model = Model1(data_bridge)
model.optimize()

Set parameter Username
Set parameter LicenseID to value 2678447
Academic license - for non-commercial use only - expires 2026-06-16
Creating variable for Interpreter Interpreter 001, Block Block 001, Session Session 001, Languages English and Spanish
Creating variable for Interpreter Interpreter 001, Block Block 001, Session Session 003, Languages English and Spanish
Creating variable for Interpreter Interpreter 002, Block Block 001, Session Session 001, Languages French and Spanish
Creating variable for Interpreter Interpreter 005, Block Block 001, Session Session 001, Languages French and Spanish
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1360P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 4 columns and 8 nonzeros
Model fingerprint: 0xa3cb0d04
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient st

In [6]:
def Model2(data):
    # Create a new model
    m = gp.Model("Model3")

    Blocks = data["Blocks"]
    Sessions = data["Sessions"]
    Sessions_blocks = data["Sessions_b"]
    Interpreters = data["Interpreters"]
    Languages = data["Languages"]
    Languages_sessions = data["Languages_s"]
    Languages_interpreters = data["Languages_i"]

    # Create variables
    x = {}
    for i in Interpreters:
        for b in Blocks:
            for s in Sessions_blocks[b]:
                for l1 in Languages_sessions[s]:
                    if l1 in Languages_interpreters[i]:
                        for l2 in Languages_sessions[s]:
                            if l2 in Languages_interpreters[i] and l2 > l1:
                                print(f"Creating variable for Interpreter {i}, Block {b}, Session {s}, Languages {l1} and {l2}")
                                # Create a binary variable for each interpreter, block, session, and language
                                x[i, b, s, l1, l2] = m.addVar(vtype=GRB.BINARY, name=f"x_{i}_{b}_{s}_{l1}_{l2}")
    
    # Constraints
    # Each interpreter can only be assigned to one language pair per Block
    for i in Interpreters:
        for b in Blocks:
            m.addConstr(gp.quicksum(x[i, b, s, l1, l2] for s in Sessions_blocks[b] for l1 in Languages_sessions[s] for l2 in Languages_sessions[s] if (i, b, s, l1, l2) in x) <= 1, name=f"OneLanguagePairPerSession_{i}_{s}")

    #Only one interpreter per language pair per session
    for b in Blocks:
        for s in Sessions_blocks[b]:
            for l1 in Languages_sessions[s]:
                for l2 in Languages_sessions[s]:
                    if l2 > l1:
                        m.addConstr(gp.quicksum(x[i, b, s, l1, l2] for i in Interpreters if (i, b, s, l1, l2) in x) <= 1, name=f"OneInterpreterPerLanguagePair_{b}_{s}_{l1}_{l2}")
    
    # Max 15 blocks per interpreter
    for i in Interpreters:
        m.addConstr(gp.quicksum(x[i, b, s, l1, l2] for b in Blocks for s in Sessions_blocks[b] for l1 in Languages_sessions[s] for l2 in Languages_sessions[s] if (i, b, s, l1, l2) in x) <= 15, name=f"MaxBlocksPerInterpreter_{i}")

    #Max 3 blocks in a row
    for i in Interpreters:
        for b in range(len(Blocks) - 3):
            m.addConstr(gp.quicksum(x[i, Blocks[b], s, l1, l2] for s in Sessions_blocks[Blocks[b]] for l1 in Languages_sessions[s] for l2 in Languages_sessions[s] if (i, Blocks[b], s, l1, l2) in x) +
                          gp.quicksum(x[i, Blocks[b + 1], s, l1, l2] for s in Sessions_blocks[Blocks[b + 1]] for l1 in Languages_sessions[s] for l2 in Languages_sessions[s] if (i, Blocks[b + 1], s, l1, l2) in x) +
                          gp.quicksum(x[i, Blocks[b + 2], s, l1, l2] for s in Sessions_blocks[Blocks[b + 2]] for l1 in Languages_sessions[s] for l2 in Languages_sessions[s] if (i, Blocks[b + 2], s, l1, l2) in x) +
                          gp.quicksum(x[i, Blocks[b + 2], s, l1, l2] for s in Sessions_blocks[Blocks[b + 2]] for l1 in Languages_sessions[s] for l2 in Languages_sessions[s] if (i, Blocks[b + 2], s, l1, l2) in x) <=3,
                          name=f"MaxThreeBlocksInARow_{i}_{b}")
    # Set objective function
    m.setObjective(gp.quicksum(x[i, b, s, l1, l2] for i in Interpreters for b in Blocks for s in Sessions_blocks[b] for l1 in Languages_sessions[s] for l2 in Languages_sessions[s] if (i, b, s, l1, l2) in x), GRB.MAXIMIZE)
    return m

In [7]:
with open(r"instances\test1.json", "r") as file:
    data = json.load(file)
model2 = Model2(data)
model2.optimize()


#print COnstraints
for c in model2.getConstrs():
    print(f"{c.constrName}: {c.Slack}")
# Print the results
for v in model2.getVars():
    if v.x > 0:
        print(f"{v.varName}: {v.x}")

Creating variable for Interpreter Interpreter 001, Block Block 001, Session Session 001, Languages English and Spanish
Creating variable for Interpreter Interpreter 001, Block Block 001, Session Session 002, Languages English and Spanish
Creating variable for Interpreter Interpreter 001, Block Block 001, Session Session 003, Languages English and Spanish
Creating variable for Interpreter Interpreter 001, Block Block 002, Session Session 004, Languages English and Spanish
Creating variable for Interpreter Interpreter 001, Block Block 003, Session Session 005, Languages English and Spanish
Creating variable for Interpreter Interpreter 001, Block Block 004, Session Session 006, Languages English and Spanish
Creating variable for Interpreter Interpreter 001, Block Block 005, Session Session 007, Languages English and Spanish
Creating variable for Interpreter Interpreter 001, Block Block 006, Session Session 008, Languages English and Spanish
Gurobi Optimizer version 12.0.2 build v12.0.2rc0

In [8]:
def Model3(data):
    # Create a new model
    m = gp.Model("Model3")

    Blocks = data["Blocks"]
    Sessions = data["Sessions"]
    Sessions_blocks = data["Sessions_b"]
    Interpreters = data["Interpreters"]
    Languages = data["Languages"]
    Languages_sessions = data["Languages_s"]
    Languages_interpreters = data["Languages_i"]

    # Create variables
    x = {}
    for i in Interpreters:
        for b in Blocks:
            for s in Sessions_blocks[b]:
                for l1 in Languages_interpreters[i]:
                    for l2 in Languages_interpreters[i]:
                        if l2 > l1:
                            print(f"Creating variable for Interpreter {i}, Block {b}, Session {s}, Languages {l1} and {l2}")
                            # Create a binary variable for each interpreter, block, session, and language
                            x[i, b, s, l1, l2] = m.addVar(vtype=GRB.BINARY, name=f"x_{i}_{b}_{s}_{l1}_{l2}")
    
    #bridge
    y = {}
    for i1 in Interpreters:
        for i2 in Interpreters:
            if i2 > i1:
                for b in Blocks:
                    for s in Sessions_blocks[b]:
                        for l_p in list(set(Languages_interpreters[i1])& set(Languages_interpreters[i2])):
                            for l1 in Languages_interpreters[i1]:
                                for l2 in Languages_interpreters[i2]:
                                    if l2 > l1 and l2 != l_p and l1 != l_p:
                                        y[i1, i2, b, s, l1, l2, l_p] = m.addVar(vtype= GRB.BINARY, name =f"y_{i1}_{i2}_{b}_{s}_{l1}_{l2}_{l_p}")
    
    LC={}
    for b in Blocks:
        for s in Sessions_blocks[b]:
            for l1 in Languages_sessions[s]:
                for l2 in Languages_sessions[s]:
                    if l2 > l1:
                        LC[b, s, l1, l2] = m.addVar(vtype=GRB.BINARY,name = f"LC_{b}_{s}_{l1}_{l2}")
    # Constraints
    # Each interpreter can only be assigned to one language pair per Block
    for i in Interpreters:
        for b in Blocks:
            m.addConstr(gp.quicksum(x[i, b, s, l1, l2] for s in Sessions_blocks[b] for l1 in Languages_interpreters[i] for l2 in Languages_interpreters[i] if (i, b, s, l1, l2) in x) <= 1, name=f"OneLanguagePairPerSession_{i}_{s}")

    #Only one interpreter per language pair per block
    for b in Blocks:
        for s in Sessions_blocks[b]:
            for l1 in Languages_interpreters[i]:
                for l2 in Languages_interpreters[i]:
                    if l2 > l1:
                        m.addConstr(gp.quicksum(x[i, b, s, l1, l2] for i in Interpreters if (i, b, s, l1, l2) in x) <= 1, name=f"OneInterpreterPerLanguagePair_{b}_{s}_{l1}_{l2}")
    
    #interpreter must be assigned to language to bridge
    for i1 in Interpreters:
        for i2 in Interpreters:
            if i2 > i1:
                for b in Blocks:
                    for s in Sessions_blocks[b]:
                        for l_p in list(set(Languages_interpreters[i1])& set(Languages_interpreters[i2])):
                            for l1 in Languages_interpreters[i1]:
                                for l2 in Languages_interpreters[i2]:
                                    if l2 > l1 and l1 in Languages_sessions[s] and l2 in Languages_sessions[s] and l1!= l_p and l2!= l_p:
                                        if (l1 < l_p):
                                            m.addConstr(y[i1, i2, b, s, l1, l2, l_p] <= x[i1, b, s, l1, l_p])                           
                                        elif (l1 > l_p):
                                            m.addConstr(y[i1, i2, b, s, l1, l2, l_p] <= x[i1, b, s, l_p, l1])
                                        if (l2 < l_p):
                                            m.addConstr(y[i1, i2, b, s, l1, l2, l_p] <= x[i2, b, s, l2, l_p])
                                        else:
                                            m.addConstr(y[i1, i2, b, s, l1, l2, l_p] <= x[i2, b, s, l_p, l2])


    #Only one bridge allowed
    
    for i1 in Interpreters:
            for b in Blocks:
                m.addConstr(gp.quicksum(y[i1, i2, b, s, l1, l2, l_p] for s in Sessions_blocks[b] for i2 in Interpreters for l1 in Languages_interpreters[i1] for l2 in Languages_interpreters[i2] for l_p in Languages if (i1, i2, b, s, l1, l2, l_p) in y) <=1)
    for i2 in Interpreters:
        for b in Blocks:
            m.addConstr(gp.quicksum(y[i1, i2, b, s, l1, l2, l_p] for s in Sessions_blocks[b] for i1 in Interpreters for l1 in Languages_interpreters[i1] for l2 in Languages_interpreters[i2] for l_p in Languages if (i1, i2, b, s, l1, l2, l_p) in y) <=1)

    # Max 15 blocks per interpreter
    for i in Interpreters:
        m.addConstr(gp.quicksum(x[i, b, s, l1, l2] for b in Blocks for s in Sessions_blocks[b] for l1 in Languages_interpreters[i] for l2 in Languages_interpreters[i] if (i, b, s, l1, l2) in x) <= 15, name=f"MaxBlocksPerInterpreter_{i}")

    #Max 3 blocks in a row
    for i in Interpreters:
        for b in range(len(Blocks) - 3):
            m.addConstr(gp.quicksum(x[i, Blocks[b], s, l1, l2] for s in Sessions_blocks[Blocks[b]] for l1 in Languages_interpreters[i] for l2 in Languages_interpreters[i] if (i, Blocks[b], s, l1, l2) in x) +
                          gp.quicksum(x[i, Blocks[b + 1], s, l1, l2] for s in Sessions_blocks[Blocks[b + 1]] for l1 in Languages_interpreters[i] for l2 in Languages_interpreters[i] if (i, Blocks[b + 1], s, l1, l2) in x) +
                          gp.quicksum(x[i, Blocks[b + 2], s, l1, l2] for s in Sessions_blocks[Blocks[b + 2]] for l1 in Languages_interpreters[i] for l2 in Languages_interpreters[i] if (i, Blocks[b + 2], s, l1, l2) in x) +
                          gp.quicksum(x[i, Blocks[b + 2], s, l1, l2] for s in Sessions_blocks[Blocks[b + 2]] for l1 in Languages_interpreters[i] for l2 in Languages_interpreters[i] if (i, Blocks[b + 2], s, l1, l2) in x) <=3,
                          name=f"MaxThreeBlocksInARow_{i}_{b}")
    
    #Language covered
    for b in Blocks:
        for s in Sessions_blocks[b]:
            for l1 in Languages_sessions[s]:
                for l2 in Languages_sessions[s]:
                    if l2 > l1:
                        m.addConstr(
                            LC[b,s, l1, l2] <=
                                gp.quicksum(y[i1, i2, b, s, l1, l2, l_p] for i1 in Interpreters for i2 in Interpreters for l_p in Languages if (i1, i2, b, s, l1, l2, l_p) in y)
                                + gp.quicksum(x[i,b,s,l1,l2] for i in Interpreters if l1 in Languages_interpreters[i] and l2 in Languages_interpreters[i]))
                        

    # Set objective function
    m.setObjective(gp.quicksum(LC[b, s, l1, l2] for b in Blocks for s in Sessions_blocks[b] for l1 in Languages_sessions[s] for l2 in Languages_sessions[s] if l1 < l2), GRB.MAXIMIZE)
    return m

In [ ]:
data_big = json.load(open(r"simplie_instances\example.json", "r"))
m3 = Model3(data_bridge)
m3.optimize()

#print results
# Print the results
for v in m3.getVars():
    if v.x > 0:
        print(f"{v.varName}: {v.x}")


Creating variable for Interpreter Interpreter 001, Block Block 001, Session Session 001, Languages English and Spanish
Creating variable for Interpreter Interpreter 001, Block Block 001, Session Session 002, Languages English and Spanish
Creating variable for Interpreter Interpreter 001, Block Block 001, Session Session 003, Languages English and Spanish
Creating variable for Interpreter Interpreter 002, Block Block 001, Session Session 001, Languages French and Spanish
Creating variable for Interpreter Interpreter 002, Block Block 001, Session Session 002, Languages French and Spanish
Creating variable for Interpreter Interpreter 002, Block Block 001, Session Session 003, Languages French and Spanish
Creating variable for Interpreter Interpreter 003, Block Block 001, Session Session 001, Languages Dutch and German
Creating variable for Interpreter Interpreter 003, Block Block 001, Session Session 002, Languages Dutch and German
Creating variable for Interpreter Interpreter 003, Block 